In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [ ]:
# Load the dataset from a CSV file
data = pd.read_csv('./shuffled_data.csv')

# Display the entire dataset
print(data)

# Print the shape of the dataset (number of rows, number of columns)
print(data.shape)

         ax    ay    az    gx     gy     gz     mx     my     mz  label
0     -0.09  0.98  0.03 -2.32  -0.55   2.14  19.86 -33.91 -26.78      3
1      0.08  1.00 -0.14  1.95  14.28   5.25  25.09 -35.69 -23.88      3
2     -0.23 -0.23 -0.97  4.88  -1.77   1.71  17.85 -15.81  14.23      2
3     -0.02  0.03 -0.99 -6.65 -11.23   7.20  22.01 -22.28  11.65      2
4      0.06  0.01  1.00  6.35  -2.87   0.92  22.95   2.32 -39.58      1
...     ...   ...   ...   ...    ...    ...    ...    ...    ...    ...
11495 -0.94  0.34 -0.07 -2.44  -8.06  -4.70 -11.05  -5.48 -18.02      4
11496  0.99 -0.02  0.05  2.93  -2.38   1.16  44.68 -11.85  -2.69      5
11497  0.16 -0.93  0.33  5.37  -3.97  10.93  23.46  18.96 -10.88      3
11498  0.99 -0.05  0.04  4.88  -1.71   2.26  44.95 -12.27  -4.13      5
11499 -0.02  0.99  0.08  9.03 -20.14 -19.35  22.24 -32.84 -30.14      3

[11500 rows x 10 columns]
(11500, 10)


In [ ]:
# Define the window size for time series data
window_size=40
 # Create windows and labels
windows = []
labels = []
sensor_cols = ['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz']

In [ ]:

# Slide a window over the data to create input sequences
for i in range(len(data) - window_size + 1):
    window = data.iloc[i:i+window_size][sensor_cols].values
    label = data.iloc[i+window_size-1]['label']
    windows.append(window)
    labels.append(label)
# Convert lists to numpy arrays
X = np.array(windows)
y = np.array(labels)

# Convert labels to zero-based indexing and one-hot encode
y = y - 1  # Convert 1-5 to 0-4
y = tf.keras.utils.to_categorical(y, num_classes=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Build the LSTM model
n_features = 9
model = Sequential([
    LSTM(16, input_shape=(window_size, n_features), return_sequences=False),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# Print the model summary
model.summary()

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 16)                  │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │              45 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,845 (7.21 KB)

 Trainable params: 1,845 (7.21 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.2494 - loss: 1.5908 - val_accuracy: 0.6799 - val_loss: 1.1144
Epoch 2/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.5365 - loss: 1.1105 - val_accuracy: 0.8528 - val_loss: 0.6064
Epoch 3/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6831 - loss: 0.7779 - val_accuracy: 0.9040 - val_loss: 0.3731
Epoch 4/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7600 - loss: 0.5980 - val_accuracy: 0.9738 - val_loss: 0.2617
Epoch 5/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8003 - loss: 0.5139 - val_accuracy: 0.9891 - val_loss: 0.1698
Epoch 6/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8737 - loss: 0.4184 - val_accuracy: 0.9935 - val_loss: 0.1168
Epoch 7/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8818 - loss: 0.3671 - val_accuracy: 0.9945 - val_loss: 0.0902
Epoch 8/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8822 - loss: 0.3466 - 

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_accuracy:.4f}")


Test Accuracy: 0.9868


In [ ]:
# Save the model
model.save('posture_model.h5')



In [ ]:

import tensorflow as tf

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable resource variables and disable lowering of tensor list ops
converter.experimental_enable_resource_variables = True
converter._experimental_lower_tensor_list_ops = False # explicitly disable lowering tensor list ops

# If using TensorFlow 2.3 or later, use the following instead:
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite builtins
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()

# Save TFLite model
with open('posture_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("\nModel saved as 'posture_model.h5' and 'posture_model.tflite'")




Saved artifact at '/tmp/tmpdiqfle2y'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  138071888316544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071888323760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880016016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880012496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880016544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880018304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880018128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880015840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880013200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138071880019008: TensorSpec(shape=(), dtype=tf.resource, name=None)

Model saved as 'postu

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat posture_model.tflite | xxd -i >> /content/model.h
!echo "};" >> /content/model.h
import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
!echo "const unsigned char model[] = {" > /content/model.h
!cat posture_model.tflite | xxd -i >> /content/model.h
!echo "};" >> /content/model.h
import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")
!echo "};" >> /content/model.h
import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")
#print(f"Header file, model.h, is {model_h_size:, } bytes.")
print(f"Header file, model.h, is {model_h_size:,} bytes.") # Original line with correct formatting
#print(f"Header file, model.h, is {model_h_size:, } bytes.")  # Problematic line
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 194,162 bytes.
Header file, model.h, is 194,162 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
Header file, model.h, is 194,165 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
Header file, model.h, is 194,165 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.


cat: gesture_model.tflite: No such file or directory
Header file, model.h, is 35 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
